# Comparison solidian tool through GCS - General Cross-Section method



In [23]:
%matplotlib widget
# %matplotlib qt

In [24]:
from bmcs_cross_section.api import MKappa, TShape, EC2, BarLayer, ReinfLayer, FabricLayer
from bmcs_beam.api import BoundaryConditions, BoundaryConfig, DeflectionProfile, BeamDesign, LoadDeflectionParamsStudy
from os.path import join
import numpy as np
import pandas as pd
import sympy as sp
import matplotlib.pyplot as plt
import bmcs_utils.api as bu

In [30]:
# Beam B-M-C-K1 - CFRP Bars

def get_dp(f_ck = None, E=None, low_kappa = None, high_kappa = None, n_kappa=None, n_load_steps=None, eps_tu=None):
    if eps_tu is None:
        eps_tu = 0.00008
    
    if n_load_steps is None:
        n_load_steps = 100
          
    if high_kappa is None:
        high_kappa = 0.00011
        
    if n_kappa is None:
        n_kappa = 200
        
    if low_kappa is None:
        low_kappa = 0
        
    mc = MKappa(low_kappa = low_kappa, high_kappa = high_kappa, n_kappa = n_kappa)

    print('Execute 2')
    f_ck = 55
    f_cd = EC2.get_f_cd(f_ck)
    E_cm = EC2.get_E_cm(f_ck)
    eps_cr = 0.000125 * f_ck/30
    eps_cy = EC2.get_eps_c2(f_ck),
    eps_cu = EC2.get_eps_cu2(f_ck),


    # mc.cs_design.matrix='EC2 with plateau'
    mc.cs_design.matrix.trait_set(
            E_cc=E_cm,
            E_ct=E_cm,
            eps_cr=eps_cr, #0.000125 * f_ck/30, # f_ctm /E,
            eps_cy=eps_cy,
            eps_cu=eps_cu,
            eps_tu=eps_tu,
            mu=0.0,
            f_cd = f_cd, # f_ck * 0.85/1.5
            factor = 1
        )
    print(f' f_ck:{f_ck:.2f}')
    print(f' f_cd:{f_cd:.2f}')
    print(f' E_cm:{E_cm:.2f}')
    print(f' eps_cr:{eps_cr:.6f}')
    print(f' eps_cy:{eps_cy[0]:.6f}')
    print(f' eps_cu:{eps_cu[0]:.6f}')

    b = 300
    h = 500
    
    mc.cross_section_shape_.B = b
    mc.cross_section_shape_.H = h

    
    bl1 = ReinfLayer(name='carbon', z=30, A=69.6, matmod='carbon')
    bl1.matmod_.trait_set(E=100000, f_t=766)
    mc.cross_section_layout.add_layer(bl1)
    print(f'Moment capacity: {(mc.M_t.max())/1e6}')

    dp = DeflectionProfile(mc=mc, n_load_steps = n_load_steps)


    dp.beam_design.beam_conf_name = BoundaryConfig.FOUR_PB
    dp.beam_design.beam_conf_name.first_load_distance = 1925
    dp.beam_design.L = 4700
    # dp.beam_design.L = 5000 # This gives perfect results
    dp.F_scale = 2/1000

    return mc

get_dp().interact()

Execute 2
 f_ck:55.00
 f_cd:31.17
 E_cm:38214.21
 eps_cr:0.000229
 eps_cy:0.002199
 eps_cu:0.003125
Moment capacity: 44.33207269447579


Output()

In [26]:
dp = get_dp()
empty = dp.mc.kappa_t[np.where(dp.mc.kappa_t < 0)]

Execute 2
 f_ck:55.00
 f_cd:31.17
 E_cm:38214.21
 eps_cr:0.000229
 eps_cy:0.002199
 eps_cu:0.003125
Moment capacity: 44.43098740640945


AttributeError: 'MKappa' object has no attribute 'mc'

In [ ]:
 print((mc.M_c_t.min())/1e6)

NameError: name 'mc' is not defined

In [ ]:
dp.mc.kappa_t

array([0.000e+00, 2.063e-07, 4.125e-07, 6.188e-07, 8.250e-07, 1.031e-06,
       1.238e-06, 1.444e-06, 1.650e-06, 1.856e-06, 2.063e-06, 2.269e-06,
       2.475e-06, 2.681e-06, 2.888e-06, 3.094e-06, 3.300e-06, 3.506e-06,
       3.713e-06, 3.919e-06, 4.125e-06, 4.331e-06, 4.538e-06, 4.744e-06,
       4.950e-06, 5.156e-06, 5.362e-06, 5.569e-06, 5.775e-06, 5.981e-06,
       6.188e-06, 6.394e-06, 6.600e-06, 6.806e-06, 7.013e-06, 7.219e-06,
       7.425e-06, 7.631e-06, 7.838e-06, 8.044e-06, 8.250e-06, 8.456e-06,
       8.663e-06, 8.869e-06, 9.075e-06, 9.281e-06, 9.488e-06, 9.694e-06,
       9.900e-06, 1.011e-05, 1.031e-05, 1.052e-05, 1.072e-05, 1.093e-05,
       1.114e-05, 1.134e-05, 1.155e-05, 1.176e-05, 1.196e-05, 1.217e-05,
       1.238e-05, 1.258e-05, 1.279e-05, 1.299e-05, 1.320e-05, 1.341e-05,
       1.361e-05, 1.382e-05, 1.403e-05, 1.423e-05, 1.444e-05, 1.464e-05,
       1.485e-05, 1.506e-05, 1.526e-05, 1.547e-05, 1.568e-05, 1.588e-05,
       1.609e-05, 1.629e-05, 1.650e-05, 1.729e-05, 